In [2]:
import pandas as pd
file_path = "/workspaces/nexus/data/world/csv/world_migration_time_1995.csv"
df = pd.read_csv(file_path)

In [3]:
df

,Unnamed: 0,ABW,AFG,AGO,ALB,ARE,ARG,ARM,ATG,AUS,...,VCT,VEN,VIR,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
0,ABW,0,0,0,0,0,1,0,2,36,...,15,337,78,0,0,0,0,0,0,0
1,AFG,0,0,0,0,501,4,0,0,5859,...,0,0,0,0,0,0,0,7,0,0
2,AGO,0,0,0,0,0,2,0,0,41,...,0,19,0,0,0,0,0,1957,21090,0
3,ALB,0,0,0,0,0,21,1,0,261,...,0,3,0,0,0,0,0,22,0,0
4,ARE,1,233,0,1,0,2,1,1,1157,...,0,6,0,83,0,0,6610,99,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,WSM,0,0,0,0,0,0,0,0,4742,...,0,0,0,0,1,0,0,0,0,0
196,YEM,0,0,0,0,31178,0,0,0,253,...,0,1,0,0,0,0,0,24,0,0
197,ZAF,0,3,1801,8,16,122,0,5,10042,...,0,41,0,14,0,0,7,0,1265,6123
198,ZMB,0,0,34358,0,96,0,0,1,934,...,0,1,0,4,0,0,0,8064,0,4249
